In [1]:
# Coordinated market: trivial system 1 for illustration
s1_cap = 100000;
d2_cap = 3000;
d3_cap = 5000;

α1 = 3.5;
α2 = 5;
α3 = 5;
α12 = 2;
α13 = 4;

using JuMP
using Gurobi

m = Model(solver = GurobiSolver(OutputFlag = 0));
@variable(m, s >= 0);
@variable(m, d2 >= 0);
@variable(m, d3 >= 0);
@variable(m, f12 >= 0);
@variable(m, f13 >= 0);

@constraint(m, node1, s == f12 + f13);
@constraint(m, node2, f12 == d2);
@constraint(m, node3, f13 == d3);
@constraint(m, s <= s1_cap);
@constraint(m, d2 <= d2_cap);
@constraint(m, d3 <= d3_cap);

@variable(m, sw);
@constraint(m, sw == (α2*d2 + α3*d3) - (α1*s) - (α12*f12 + α13*f13))
@objective(m, Max, sw);
solve(m);
ϕs = (-getdual(node1)-α1)*getvalue(s);
ϕd2 = (α2 + getdual(node2))*getvalue(d2);
ϕd3 = (α3 + getdual(node3))*getvalue(d3);
ϕl = getvalue(f12)*(-getdual(node2)+getdual(node1)-α12) + getvalue(f13)*(-getdual(node3)+getdual(node1)-α13);

println("Social Welfare: ",getvalue(sw));
println("Supplier's Profit: ", ϕs);
println("Customer2's Profit: ", ϕd2);
println("Customer3's Profit: ", ϕd3);
println("Transportation Provider's Profit: ", ϕl);
println("Clearing prices: ", -getdual(node1),", ",-getdual(node2),", ", -getdual(node3))

Academic license - for non-commercial use only
Social Welfare: 0.0
Supplier's Profit: 0.0
Customer2's Profit: 0.0
Customer3's Profit: 0.0
Transportation Provider's Profit: -0.0
Clearing prices: 3.5, 5.0, 5.0


In [8]:
# Coordinated market: trivial system 2 for illustration
s1_cap = 5000;
d2_cap = 1e20;

α1 = -6;
α2 = -1.5;
α12 = 5;

using JuMP
using Gurobi

m = Model(solver = GurobiSolver(OutputFlag = 0));
@variable(m, s >= 0);
@variable(m, d2 >= 0);
@variable(m, f12 >= 0);

@constraint(m, node1, s == f12);
@constraint(m, node2, f12 == d2);
@constraint(m, s <= s1_cap);
@constraint(m, d2 <= d2_cap);

@variable(m, sw);
@constraint(m, sw == (α2*d2) - (α1*s) - (α12*f12))
@objective(m, Max, sw);
solve(m);
ϕs = (-getdual(node1)-α1)*getvalue(s);
ϕd2 = (α2 + getdual(node2))*getvalue(d2);
ϕl = getvalue(f12)*(-getdual(node2)+getdual(node1)-α12);

println("Social Welfare: ",getvalue(sw));
println("Supplier's Profit: ", ϕs);
println("Customer2's Profit: ", ϕd2);
println("Transportation Provider's Profit: ", ϕl);
println("Clearing prices: ", -getdual(node1),", ",-getdual(node2))

Academic license - for non-commercial use only
Social Welfare: 0.0
Supplier's Profit: -0.0
Customer2's Profit: 0.0
Transportation Provider's Profit: 0.0
Clearing prices: -6.5, -1.5


In [51]:
# Coordinated market: trivial system 3 for illustration
s1_cap = 10000;
ξ2_cap = 8000;
d3_cap = 1e20;
d4_cap = 1e20;

α1 = -10;
α2 = 0;
α3 = -30;
α4 = 0;
α12 = 5;
α23 = 0;
α24 = 0;

using JuMP
using Gurobi
m = Model(solver = GurobiSolver());
@variables m begin
    s1>=0;
    d3>=0;
    d4>=0;
    f12>=0;
    f23>=0;
    f24>=0;
    x1;
    x2;
    x3;
end

@constraints m begin
    node1, s1 == f12
    node2_1, f12 + x1 == 0;
    node2_2, x2 == f23;
    node2_3, x3 == f24;
    node3, f23 == d3;
    node4, f24 == d4;
    x2 == -0.01*x1;
    x3 == -0.99*x1;
    s1 <= s1_cap;
    d3 <= d3_cap;
    d4 <= d4_cap;
    x1 >= -ξ2_cap;
end

@variable(m, sw);
@constraint(m, sw == (α3*d3+α4*d4) - (α1*s1) + (α2*x1) - (α12*f12+α23*f23+α24*f24))
@objective(m, Max, sw);

solve(m);
ϕs = (-getdual(node1)-α1)*getvalue(s1);
ϕd3 = (α3 + getdual(node3))*getvalue(d3);
ϕd4 = (α4 + getdual(node4))*getvalue(d4);
ϕt2 = (-0.99*getdual(node2_3) - 0.01*getdual(node2_2) + getdual(node2_1) -α2)*(-getvalue(x1));
ϕl = getvalue(f23)*(-getdual(node3)+getdual(node2_2)-α23) + getvalue(f24)*(-getdual(node4)+getdual(node2_3)-α24) + getvalue(f12)*(-getdual(node2_1)+getdual(node1)-α12);

println("Social Welfare: ",getvalue(sw));
println("Supplier's Profit: ", ϕs);
println("Customer3's Profit: ", ϕd3);
println("Customer4's Profit: ", ϕd4);
println("Transportation Provider's Profit: ", ϕl);
println("Technology Provider's Profit: ", ϕt2);
println("Clearing prices: ", -getdual(node1),", ",-getdual(node2_1),", ",-getdual(node2_2),", ",-getdual(node2_3),", ", -getdual(node3),", ", -getdual(node4));

Academic license - for non-commercial use only
Optimize a model with 13 rows, 10 columns and 24 nonzeros
Coefficient statistics:
  Matrix range     [1e-02, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+03, 1e+20]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 13 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7600000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  3.760000000e+04
Social Welfare: 37600.0
Supplier's Profit: 0.0
Customer3's Profit: 0.0
Customer4's Profit: 0.0
Transportation Provider's Profit: 0.0
Technology Provider's Profit: 37600.0
Clearing prices: -10.0, -5.0, -30.0, 0.0, -30.0, 0.0
